In [2]:
import json
from datetime import datetime
from os import path

import pandas as pd
import requests

# Binance API url
root_binance_url = 'https://api.binance.com/api/v1/klines'
symbol_url = '?symbol='
interval_url = '&interval='
start_time_url = '&startTime='
end_time_url = '&endTime='
# limit is max 500 records, max 1200 requests/minute

symbols = ['EOSBTC', 'TRXBTC', 'ONTBTC']
# intervals: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, ...
intervals = ['5m', '15m', '30m', '1h', '4h', '1d']
end_date_time = '20.06.2018 00:00:00'


def get_data(symbol, interval):
    # check is data file exists
    fname = 'binance_' + symbol + '_' + interval + '.json'
    if not path.isfile(fname):
        # from first available timestamp
        url = root_binance_url + symbol_url + symbol + interval_url + interval + start_time_url + '0'
        data = json.loads(requests.get(url).text)

        # new start time is the previous end timestamp, 500 is the limit/max
        start_time = data[-1][0]
        end_time = convert_timestamp(end_date_time)
        while start_time < end_time:
            url = root_binance_url + symbol_url + symbol + interval_url + interval + start_time_url + str(start_time)
            data_new = json.loads(requests.get(url).text)
            # omit the first element as it is equal to the last on the previous list
            data = data + data_new[1:]
            start_time = data[-1][0]

        # save it to file
        with open(fname, 'w') as f:
            json.dump(data, f, sort_keys=True, indent=4, ensure_ascii=False)
    else:
        print('Data for {}, interval {} already exists...'.format(symbol, interval))


def convert_timestamp(date_time):
    dt = datetime.strptime(date_time, '%d.%m.%Y %H:%M:%S')
    millis_time = dt.timestamp() * 1000

    return int(millis_time)


def prepare_data(fname):
    df = pd.read_json(fname)

    # remove any rows with null values
    df = df.dropna()

    # only need [Open Time, Open, High, Low, Close, Volume] columns
    df.drop(df.columns[6:], axis=1, inplace=True)
    col_names = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.columns = col_names

    # remove rows after end time
    end_time = convert_timestamp(end_date_time)
    df = df[df['Open Time'] <= end_time]

    # sort by descending date, so that last point presents 15th June 2018
    df = df.sort_values(by='Open Time')

    # calc volatility
    df = df.assign(Volatility=lambda x: (x['High'] - x['Low']) / x['Open'])

    # Close price, Volume and Volatility are the only columns needed for next step - TA calculation
    df.drop(df.columns[:4], axis=1, inplace=True)

    return df


def plot_data(dataframe):
    dataframe.plot()


coin_data = []
for sym in symbols:
    for t in intervals:
        print('\n\nGetting data for {}, interval {}'.format(sym, t))
        get_data(sym, t)

        print('\nProcessing data for {}'.format(sym))
        filename = 'binance_' + sym + '_' + t + '.json'
        coin_data.append(prepare_data(filename))

# plot_data(coin_data)




Data for EOSBTC, interval 5m


Data for EOSBTC, interval 15m


Data for EOSBTC, interval 30m


Data for EOSBTC, interval 1h


Data for EOSBTC, interval 4h


Data for EOSBTC, interval 1d


Data for TRXBTC, interval 5m


Data for TRXBTC, interval 15m


Data for TRXBTC, interval 30m


Data for TRXBTC, interval 1h


Data for TRXBTC, interval 4h


Data for TRXBTC, interval 1d


Data for ONTBTC, interval 5m


Data for ONTBTC, interval 15m


Data for ONTBTC, interval 30m


Data for ONTBTC, interval 1h


Data for ONTBTC, interval 4h


Data for ONTBTC, interval 1d


NameError: name 'symbols' is not defined